## 2019 Bushfires in Pechey

Pechey is a small rural locality in the Toowoomba Region, Queensland. The New England Highway passes through the centre of the town and the Pechey State Forest occupies the entire south east corner of the area.

On the 21st of November 2019 the ABC news service reported: "Four homes have been lost in a bushfire that has been burning for 10 days in the Pechey area on Queensland's Darling Downs, authorities say."

<img src="data/Pechey_Satellite.png" alt="drawing" width="400" align="left"/>

### Your task:

You work at InsFire Inc. an new insurance company specialised in using remote sensing data to automatically process the claims of its clients. You are in charge of developing the algorithms that detect burnt areas and are interested in validating this algorithms for places were fires have been reported.

You have heard in the news about the Pechey fire and want to use your algorithm to detect and measure the total area burnt in this fire event.

### Load packages

You start by loading the usual Python libraries to start working on this project.

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

import datacube
from datacube.utils import masking

### Load data

You request Sentinel 2 data around of November 2019.

In [ ]:
dc = datacube.Datacube(app='bushfires')

query = {'lat': (-27.31, -27.35),
         'lon': (152.06 , 152.16),
         'output_crs': 'EPSG:3577',
         'resolution': (-20, 20),
         'time': ('2019-10-15', '2019-11-22')}

pechey = dc.load(product='s2b_ard_granule', group_by='solar_day', **query)

pechey = masking.mask_invalid_data(pechey)

#display RGB colors
pechey[['nbart_red', 'nbart_green', 'nbart_blue']].to_array().plot.imshow(robust=True,col='time')

You can clearly see the effects of the fires in this two images. The second one even shows the smoke of the fires which seem to be still active on the 16th of November.


# The algorithm behind InsFire Inc.

The Normalized Burn Ratio (NBR) is an index designed to highlight burnt areas in large fire zones. The formula is similar to NDVI, except that the formula combines the use of both near infrared (NIR) and shortwave infrared (SWIR) wavelengths.

Healthy vegetation shows a very high reflectance in the NIR, and low reflectance in the SWIR portion of the spectrum the opposite of what is seen in areas devastated by fire. Recently burnt areas demonstrate low reflectance in the NIR and high reflectance in the SWIR, i.e. the difference between the spectral responses of healthy vegetation and burnt areas reach their peak in the NIR and the SWIR regions of the spectrum.

<figure>
  <img src="data/Spectral_responses.jpg" alt="drawing" width="400" align="left"/>
  <figcaption> service</figcaption>
</figure>

Comparison of the spectral response of healthy vegetation and burned areas. Source: U.S. Forest

To benefit from the magnitude of spectral difference, NBR uses the ratio between NIR and SWIR bands, according to the formula shown below. A high NBR value indicates healthy vegetation while a low value indicates bare ground and recently burnt areas. Non-burnt areas are normally attributed to values close to zero.

$NBR = \frac{nir-swir2}{nir+swir2}$

### Computing and Visualising NBR indexes

For the returned Dataset we calculate the NBR index for both times using the previous formula:

In [ ]:
nbr = (pechey['nbar_nir_1']-pechey['nbart_swir_3'])/(pechey['nbar_nir_1']+pechey['nbart_swir_3'])
nbr.plot.imshow(col='time', col_wrap=2, cmap='seismic')

The NBR indexes show a clear decrease in NBR values between these two days, which are separated only by 10 days. This has to be the effect of the fire to have such large change in just 10 days.

### Temporal change in NBR

There are still some areas which show low values of NBR before the fires, so you decide to calculate the difference of the NBR values between both days.

In [ ]:
(nbr.isel(time=1)-nbr.isel(time=0)).plot.imshow(cmap='seismic')

This seems to be a much better representation of the area affected by the fires. The very low values around -1 probably indicate active fires with high temperatures that show up on the near infrared channel.

### Categorical Representation of NBR Differences

You decide to create a map to represent these NBR values using 6 ranges represented with different colours. You convert the NBR difference into a simple classification map with 6 thresholds.

In [ ]:
nbr_diff = (nbr.isel(time=1)-nbr.isel(time=0)).values

diff = np.zeros((nbr_diff.shape))

diff[(nbr_diff<0)*(nbr_diff>=-0.1)] = 1
diff[(nbr_diff>-0.2)*(nbr_diff<=-0.1)] = 2
diff[(nbr_diff>-0.3)*(nbr_diff<=-0.2)] = 3
diff[(nbr_diff>-0.4)*(nbr_diff<=-0.3)] = 4
diff[(nbr_diff>-0.5)*(nbr_diff<=-0.4)] = 5
diff[nbr_diff<-0.5] = 6

plt.figure(figsize=(10,6))
plt.imshow(diff,cmap='gist_heat_r')
plt.colorbar()

### Custom color coding

But you are not happy with the colour map used by matplotlib and want to create a custom one with colours that represent better each category.

In [ ]:
from matplotlib.colors import ListedColormap

plt.figure(figsize=(20,12))
cMap = ListedColormap(['olive','yellowgreen','white', 'yellow','orange','orangered','purple'])
plt.imshow(diff,cmap=cMap)
cbar = plt.colorbar()
cbar.ax.set_yticklabels(['Enhanced Regrowth, high','Enhanced Regrowth, low','Unburned','Low Severity','Moerate-low Severity','Miderate-high Severity','High Severity'],fontsize=16)
plt.xticks([])
plt.yticks([])

### Calculating total burned area

Finally for this area you want to come up with an estimate of the total burned area. You start by creating a binary mask differentiating the burned from the non-burned areas.

Can you come up with a value that allows you differentiate areas affected by the fire?

In [ ]:
plt.imshow(nbr_diff < #Your value here#)

Finally, once you are happy with your value, you want to calculate the total burned area in hectares for which you need to:
    
1. Get the sum of all the pixels labelled with `True`. _Hint: You might find the documentation of the `np.count_nonzero()` function useful here_
2. Going back to your original DEA query you'll notice each pixel covers a 20x20 metres area, which helps you get the number of square metres.
3. One Hectare is 10,000 m2

### Other locations

In the last month there has been many bushfires in NSW and Queensland. You'll need some more locations to test your algorithm so you decide to search in the news for other locations that have suffered from the fire.

_This website contains updated information about the location and state of bushfires in Australia:
http://anuwald.science/afms_